<a href="https://colab.research.google.com/github/toshit11/yolov5-based-on-ultralytics/blob/master/Custom_yolo5l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies

%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
torch.cuda.empty_cache()

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


Setup complete. Using torch 1.5.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/') # mounting the drive on the google colab
!ls "/content/drive/My Drive"
!unzip "/content/drive/My Drive/data.zip" -d "/content/yolov5_traffic_lights/"
!unzip "/content/drive/My Drive/sample_images.zip" -d "/content/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
 00013.m2ts
 00066.mp4
 1553766661956902838117.jpg
 298473401.pdf
 99991596.pdf
 99991890.pdf
'AMU notes 2.pdf'
 analy
'Automatic Lighting & Accident Detection System(Final Report).pdf'
 CE111.pdf
'Certificate '
 chapter05.pdf
 classifier.pt
 Classroom
'Colab Notebooks'
 Cost_Estimation.gsheet
'Coursera 2PSKPLYHFY7S (1).pdf'
'Coursera 2PSKPLYHFY7S (2).pdf'
'Coursera 2PSKPLYHFY7S.pdf'
'Coursera AWN49FPZVX3N (1).pdf'
'Coursera AWN49FPZVX3N (2).pdf'
'Coursera AW

In [ ]:
#!mv "/content/train.yaml"  "/content/yolov5/data" # transfering the train yaml file into the yolov5 folder

In [ ]:
import yaml
with open("/content/yolov5_traffic_lights/data/traffic_lights.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print(num_classes)

13


In [ ]:
with open('/content/yolov5/models/custom_yolov5l.yaml', 'w') as f:
  # parameters
  f.write('nc: ' + num_classes + '\n')
  #f.write('nc: ' + str(len(class_labels)) + '\n')
  f.write('depth_multiple: 1'  + '\n') # model depth multiple
  f.write('width_multiple: 1'  + '\n')  # layer channel multiple
  f.write('\n')
  f.write('anchors:' + '\n')
  f.write('  - [10,13, 16,30, 33,23] ' + '\n')
  f.write('  - [30,61, 62,45, 59,119]' + '\n')
  f.write('  - [116,90, 156,198, 373,326] ' + '\n')
  f.write('\n')

  f.write('backbone:' + '\n') 
  # [from, number, module, args]
  f.write(' [[-1, 1, Focus, [64, 3]],' +'\n'),  # 0-P1/2
  f.write(' [-1, 1, Conv, [128, 3, 2]],'+'\n'),  # 1-P2/4
  f.write('  [-1, 3, BottleneckCSP, [128]],'+'\n'),
  f.write('  [-1, 1, Conv, [256, 3, 2]],'+'\n'),  # 3-P3/8
  f.write('  [-1, 9, BottleneckCSP, [256]],'+'\n'),
  f.write('  [-1, 1, Conv, [512, 3, 2]],'+'\n'),  # 5-P4/16
  f.write('  [-1, 9, BottleneckCSP, [512]],'+'\n'),
  f.write('  [-1, 1, Conv, [1024, 3, 2]],'+'\n'), # 7-P5/32
  f.write('  [-1, 1, SPP, [1024, [5, 9, 13]]],'+'\n'),
  f.write(']'+'\n')

# YOLOv5 head
  f.write('head:'+'\n')
  f.write('  [[-1, 3, BottleneckCSP, [1024, False]],'+'\n'),  # 9

  f.write('   [-1, 1, Conv, [512, 1, 1]],'+'\n'),
  f.write('   [-1, 1, nn.Upsample, [None, 2, "nearest"]],'+'\n'),
  f.write('   [[-1, 6], 1, Concat, [1]],'+'\n'),  # cat backbone P4
  f.write('   [-1, 3, BottleneckCSP, [512, False]],'+'\n'),  # 13

  f.write('   [-1, 1, Conv, [256, 1, 1]],'+'\n'),
  f.write('   [-1, 1, nn.Upsample, [None, 2, "nearest"]],'+'\n'),
  f.write('   [[-1, 4], 1, Concat, [1]],'+'\n'),  # cat backbone P3
  f.write('   [-1, 3, BottleneckCSP, [256, False]],'+'\n'),
  f.write('   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],'+'\n'),  # 18 (P3/8-small)

  f.write('   [-2, 1, Conv, [256, 3, 2]],'+'\n'),
  f.write('   [[-1, 14], 1, Concat, [1]],'+'\n'),  # cat head P4
  f.write('   [-1, 3, BottleneckCSP, [512, False]],'+'\n'),
  f.write('   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],'+'\n'),  # 22 (P4/16-medium)

  f.write('   [-2, 1, Conv, [512, 3, 2]],'+'\n'),
  f.write('   [[-1, 10], 1, Concat, [1]],'+'\n'),  # cat head P5
  f.write('   [-1, 3, BottleneckCSP, [1024, False]],'+'\n'),
  f.write('   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],'+'\n'),  # 26 (P5/32-large)

  f.write('   [[], 1, Detect, [nc, anchors]],'+'\n' ),  # Detect(P5, P4, P3)
  f.write('  ]'+'\n')



In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 16 --epochs 1200 --data '/content/yolov5_traffic_lights/data/traffic_lights.yaml' --cfg ./models/custom_yolov5l.yaml --weights '' --name yolov5s_results --nosave --cache # starting the training on bosch dataset for image size of 640

/content/yolov5
Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/custom_yolov5l.yaml', data='/content/yolov5_traffic_lights/data/traffic_lights.yaml', device='', epochs=1200, evolve=False, img_size=[640], multi_scale=False, name='yolov5s_results', noautoanchor=False, nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-07-06 11:31:29.502776: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Start Tensorboard with 